# Cloud sandbox powered by FIWARE

## About the open source GPL3 license and copyright for this product

Copyright © 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Switch to the Python 3.9 Kernel

When opening this Jupyter Notebook in VSCode, you will need to switch to the Jupyter 3.9 Kernel. 

## Deploy the IoT Agent JSON in the OpenShift Developer Sandbox

A FIWARE IoT Agent like IoT Agent JSON is required for smart device registration of vehicle traffic cameras and pedestrian traffic cameras in the project. 

## Create an IoT Agent message broker secret to connect to RabbitMQ 

In [ ]:
%%bash
oc create secret generic iotagent-json-message-broker-secret \
  --from-literal password="$(oc get secret/rabbitmq -o jsonpath={.data.rabbitmq-password} | base64 -d)"
echo DONE

## Create an IoT Agent secret to connect to MongoDB 

In [ ]:
%%bash
oc create secret generic iotagent-json-mongodb-secret \
  --from-literal password="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 15)"
echo DONE

## Run create database scripts on MongoDB

In [ ]:
%%bash
oc exec $(oc get pod -l 'app.kubernetes.io/name=mongodb' -o name) -- \
  sh -c 'mongosh -u "$MONGODB_ROOT_USER" -p "$MONGODB_ROOT_PASSWORD" --eval '"'"'
  db = db.getSiblingDB("iotagentjson"); 
  db.createUser({ user: "iotagent", pwd: "'"$(oc get secret/iotagent-json-mongodb-secret -o jsonpath={.data.password} | base64 -d)"'", roles: [{ role: "dbOwner", db: "iotagentjson"}]}); 
  db = db.getSiblingDB("iotagentjson"); 
  db.createCollection("devices"); 
  db.devices.createIndex({"_id.service": 1, "_id.id": 1, "_id.type": 1}); 
  db.devices.createIndex({"_id.type": 1}); 
  db.devices.createIndex({"_id.id": 1}); 
  db.createCollection("groups"); 
  db.groups.createIndex({"_id.resource": 1, "_id.apikey": 1, "_id.service": 1}); 
  db.groups.createIndex({"_id.type": 1}); 
  '"'"''
echo DONE

## Deploy the iotagent-json Deployment

In [ ]:
%%bash
ansible-playbook playbooks/deploy-iotagent-json.yaml \
  -e "IOTA_CB_HOST=scorpiobroker" \
  -e "IOTA_CB_PORT=9090" \
  -e "IOTA_HOST=iotagent-json" \
  -e "IOTA_NORTH_PORT=4041" \
  -e "IOTA_HTTP_PORT=7896" \
  -e "IOTA_LOG_LEVEL=INFO" \
  -e "IOTA_JSON_LD_CONTEXT=https://raw.githubusercontent.com/computate-org/computate.org-static/main/fiware/context.jsonld,https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld" \
  -e "IOTA_MONGO_HOST=mongodb" \
  -e "IOTA_MONGO_PORT=27017" \
  -e "IOTA_MONGO_USER=iotagent" \
  -e "IOTA_MONGO_SECRET_NAME=iotagent-json-mongodb-secret" \
  -e "IOTA_MONGO_SECRET_KEY=password" \
  -e "IOTA_MONGO_DB=iotagentjson" \
  -e "IOTA_AMQP_HOST=rabbitmq" \
  -e "IOTA_AMQP_PORT=5672" \
  -e "IOTA_AMQP_USERNAME=user" \
  -e "IOTA_AMQP_SECRET_NAME=iotagent-json-message-broker-secret" \
  -e "IOTA_AMQP_SECRET_KEY=password" \
  -e "IOTA_LIMITS_CPU=1" \
  -e "IOTA_LIMITS_MEMORY=1Gi" \
  -e "IOTA_REQUESTS_CPU=200m" \
  -e "IOTA_REQUESTS_MEMORY=500Mi" \
  -e "IOTAGENT_NAMESPACE=$(oc project -q)"
echo DONE

### View IoT Agent JSON pod details
After deploying IoT Agent JSON with helm, it will take a minute before the IoT Agent JSON pod is up and running. Run the command below until the IoT Agent JSON pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc get pod -l app.kubernetes.io/instance=iotagent-json
oc wait pod -l app.kubernetes.io/instance=iotagent-json --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/instance=iotagent-json
echo DONE

### View IoT Agent JSON pod logs
If your IoT Agent JSON pod does not reach the STATUS Running, you can run the command below to view the pod logs of IoT Agent JSON and check for other errors that may have occured. 

In [ ]:
%%bash
oc logs -l app.kubernetes.io/name=iotagent-json
echo DONE

## Next...
If you have successfully ran all of the commands above, congratulations, you are ready to move on to the next notebook in the course. 

- If you have additional questions or issues, please [create an issue for the course here](https://github.com/computate-org/computate/issues). 
- Otherwise, please continue to the next document [10-deploy-keycloak.ipynb](10-deploy-keycloak.ipynb). 
